# Exploratory Data Analysis

# Task
A client is thinking of setting up an Airbnb business in Bangkok, he has tasked me with analyzing Bangkok's existing Airbnbs to help him succeed.
Data sourced from: https://insideairbnb.com

In [105]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('/Users/joeytan/Documents/Data Analyst/Portfolio/PortfolioProject1_Python-Airbnb/Datasets/Cleaned Data/listings_cleaned.csv')
df2 = pd.read_csv('/Users/joeytan/Documents/Data Analyst/Portfolio/PortfolioProject1_Python-Airbnb/Datasets/Cleaned Data/calendar_cleaned.csv')

In [107]:
df1.info()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17447 entries, 0 to 17446
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           17447 non-null  int64  
 1   host_response_time           16598 non-null  object 
 2   host_response_rate           16598 non-null  object 
 3   host_acceptance_rate         16451 non-null  object 
 4   host_is_superhost            16384 non-null  object 
 5   host_has_profile_pic         17447 non-null  object 
 6   host_identity_verified       17447 non-null  object 
 7   neighbourhood_cleansed       17447 non-null  object 
 8   latitude                     17447 non-null  float64
 9   longitude                    17447 non-null  float64
 10  property_type                17447 non-null  object 
 11  room_type                    17447 non-null  object 
 12  accommodates                 17447 non-null  int64  
 13  bathrooms       

# Seasonal Demand
-- Price by month\
-- Occupancy rate by month

In [109]:
df2['date'] = pd.to_datetime(df2['date'])
df2['month'] = df2['date'].dt.to_period('M')

avg_price_by_month = df2.groupby(['month'])['price'].mean().round().reset_index().rename(columns={'price': 'avg_price'})
avg_price_by_month

,month,avg_price
0,2024-12,1578.0
1,2025-01,1579.0
2,2025-02,1579.0
3,2025-03,1578.0
4,2025-04,1578.0
5,2025-05,1578.0
6,2025-06,1578.0
7,2025-07,1578.0
8,2025-08,1578.0
9,2025-09,1578.0


In [110]:
avg_occupancy_by_month = (
    df2.groupby('month')['available'] 
    .agg(lambda x: (x == 'f').mean() * 100) 
    .round(1)
    .reset_index()
    .rename(columns={'available': 'occupancy_rate(%)'})
)

avg_occupancy_by_month

,month,occupancy_rate(%)
0,2024-12,80.7
1,2025-01,45.0
2,2025-02,25.7
3,2025-03,14.3
4,2025-04,18.9
5,2025-05,16.6
6,2025-06,19.4
7,2025-07,31.0
8,2025-08,30.8
9,2025-09,35.0


# Room Types
-- Price by room type\
-- Occupancy by room type\
-- Price by number of bedrooms\
-- Occupancy by number of bedrooms\
-- Price by capacity\
-- Occupancy by capacity

In [112]:
avg_prices_by_type = df1.groupby('room_type')['price'].mean().reset_index().sort_values(['price'],ascending=False)
    
avg_prices_by_type.rename(columns={'price': 'avg_price'})

,room_type,avg_price
0,Entire home/apt,2096.308824
1,Hotel room,2011.989130
2,Private room,1570.780909
3,Shared room,583.469136


In [113]:
avg_occupancy_by_room_type = (
    df1.groupby('room_type', as_index=False)
    .agg(avg_occupancy_rate=('availability_365', 
                           lambda x: (365 - x).mean()/365 * 100))
    .round(1)
    .sort_values('avg_occupancy_rate', ascending=False)
)

avg_occupancy_by_room_type

,room_type,avg_occupancy_rate
0,Entire home/apt,35.0
2,Private room,25.8
1,Hotel room,22.9
3,Shared room,16.7


In [114]:
avg_prices_by_rooms =df1.groupby(['bedrooms'])['price'].mean().round().reset_index().sort_values(['price'],ascending=False)
avg_prices_by_rooms

,bedrooms,price
5,5.0,3966.0
4,4.0,3472.0
3,3.0,3337.0
15,30.0,2949.0
2,2.0,2906.0
6,6.0,2567.0
9,9.0,1908.0
11,12.0,1771.0
1,1.0,1712.0
0,0.0,1398.0


In [115]:
# interesting... there are listings with 0 bedrooms, I'm guessing those are hostels with dormitories

avg_occupancy_by_rooms = (
    df1.groupby(df1['bedrooms'])['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index().rename(columns={'availability_365':'occupancy rate'}))

avg_occupancy_by_rooms

,bedrooms,occupancy rate
0,9.0,75.5
1,50.0,43.0
2,2.0,36.3
3,0.0,34.9
4,3.0,34.0
5,1.0,31.1
6,5.0,29.5
7,4.0,28.7
8,20.0,26.0
9,16.0,23.6


In [116]:
avg_prices_by_cap =df1.groupby(['accommodates'])['price'].mean().round().reset_index().sort_values(['price'],ascending=False)
avg_prices_by_cap

,accommodates,price
14,15,4814.0
12,13,4212.0
10,11,3799.0
6,7,3386.0
8,9,3351.0
9,10,3320.0
11,12,3262.0
7,8,3155.0
4,5,2938.0
5,6,2912.0


In [117]:
avg_occupancy_by_cap = (
    df1.groupby(df1['accommodates'])['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index()
    .rename(columns={'bedrooms': 'capacity','availability_365': 'avg_occupancy_rate(%)'})
)
avg_occupancy_by_cap

,accommodates,avg_occupancy_rate(%)
0,7,42.6
1,11,37.9
2,5,36.8
3,4,36.2
4,3,35.1
5,14,33.2
6,6,32.3
7,8,31.4
8,2,30.4
9,10,30.0


# Neighbourhood
-- Price by neighbourhood\
-- Occupancy by neighbourhood\
-- Density of neighbourhood\
-- Location scores by neighbourhood

In [119]:
prices_by_neighbourhood = (df1.groupby(['neighbourhood_cleansed'])
                           ['price'].mean().round().reset_index()
                           .sort_values(['price'],ascending=False)
                          .rename(columns={'price':'avg_price'}))

prices_by_neighbourhood

,neighbourhood_cleansed,avg_price
29,Parthum Wan,2640.0
8,Bang Rak,2425.0
47,Vadhana,2416.0
39,Samphanthawong,2291.0
18,Huai Khwang,2251.0
22,Khlong Toei,2222.0
37,Ratchathewi,2100.0
34,Pom Prap Sattru Phai,2053.0
33,Phra Nakhon,1970.0
41,Sathon,1779.0


In [120]:
avg_occupancy_by_neighborhood = (
    df1.groupby(['neighbourhood_cleansed'])['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index()
    .rename(columns={'availability_365': 'avg_occupancy_rate(%)'})
)
avg_occupancy_by_neighborhood

,neighbourhood_cleansed,avg_occupancy_rate(%)
0,Taling Chan,45.8
1,Yan na wa,40.3
2,Vadhana,38.4
3,Chom Thong,37.0
4,Sathon,36.2
5,Ratchathewi,35.5
6,Parthum Wan,35.3
7,Bang Na,35.0
8,Khlong Toei,34.6
9,Bang Sue,34.4


In [121]:
neighbourhood_density = (df1.groupby(['neighbourhood_cleansed'])
                         ['id'].nunique().reset_index()
                         .sort_values(by='id',ascending = False)
                        .rename(columns={'id':'listings_count'}))

neighbourhood_density

,neighbourhood_cleansed,listings_count
47,Vadhana,2796
22,Khlong Toei,2468
18,Huai Khwang,2065
37,Ratchathewi,1046
41,Sathon,832
32,Phra Khanong,802
33,Phra Nakhon,660
8,Bang Rak,583
13,Chatu Chak,543
42,Suanluang,513


In [122]:
neighbourhood_by_review_scores=(df1.groupby(['neighbourhood_cleansed'])['review_scores_location']
                                .mean().round(2).reset_index().sort_values(['review_scores_location'], ascending=False))

neighbourhood_by_review_scores

,neighbourhood_cleansed,review_scores_location
8,Bang Rak,4.80
29,Parthum Wan,4.79
43,Taling Chan,4.79
33,Phra Nakhon,4.77
39,Samphanthawong,4.75
11,Bangkok Yai,4.75
47,Vadhana,4.74
21,Khlong San,4.73
46,Thung khru,4.73
34,Pom Prap Sattru Phai,4.71


# Reviews
-- Occupancy by review scores\
-- Occupancy by number of reviews

In [124]:
avg_occupancy_by_scores = (
    df1.groupby(round(df1['review_scores_rating']))['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index()
    .rename(columns={'availability_365': 'avg_occupancy_rate(%)'})
)
avg_occupancy_by_scores

,review_scores_rating,avg_occupancy_rate(%)
0,5.0,37.0
1,4.0,31.8
2,2.0,29.2
3,1.0,27.2
4,3.0,26.6


In [125]:
avg_occupancy_by_no_of_reviews = (
    df1.groupby(round(df1['number_of_reviews']/100)*100)['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index()
    .rename(columns={'number_of_reviews': 'number_of_reviews(100s)','availability_365': 'avg_occupancy_rate(%)'})
)
avg_occupancy_by_no_of_reviews

,number_of_reviews(100s),avg_occupancy_rate(%)
0,100.0,40.8
1,200.0,39.1
2,300.0,36.8
3,400.0,32.4
4,0.0,30.9
5,500.0,24.0
6,600.0,22.6
7,700.0,9.3
8,1400.0,9.0
9,1000.0,8.1


# Superhosts
-- Occupancy rate superhost vs regular host\
-- What contributes to superhost\
---- Response time\
---- Response rate\
---- Review scores\
---- Host is verified\
---- Host has profile photo

In [127]:
avg_occupancy_by_super = (
    df1.groupby(df1['host_is_superhost'])['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index()
    .rename(columns={'availability_365': 'avg_occupancy_rate(%)'})
)
avg_occupancy_by_super

,host_is_superhost,avg_occupancy_rate(%)
0,t,37.1
1,f,28.6


In [128]:
superhost = (
    df1[df1['host_is_superhost'] == 't']
    .groupby('host_response_time')
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('Superhost (%)')
)

non_superhost = (
    df1[df1['host_is_superhost'] == 'f']
    .groupby('host_response_time')
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('non_superhost(%)')
)

pd.concat([superhost, non_superhost], axis=1).sort_values(['Superhost (%)'], ascending = False)

,Superhost (%),non_superhost(%)
host_response_time,,
within an hour,88.3,70.9
within a few hours,10.4,13.2
within a day,1.3,5.7
a few days or more,0.0,10.2


In [129]:
df1['host_response_rate'] = df1['host_response_rate'].replace('%', '', regex=True).astype(float)

superhost = (
    df1[df1['host_is_superhost'] == 't']
    .groupby(round(df1['host_response_rate']/10)*10)
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('Superhost (%)')
)

non_superhost = (
    df1[df1['host_is_superhost'] == 'f']
    .groupby(round(df1['host_response_rate']/10)*10)
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('non_superhost(%)')
)

pd.concat([superhost, non_superhost], axis=1).sort_values(['Superhost (%)'],ascending = False)

,Superhost (%),non_superhost(%)
host_response_rate,,
100.0,95.2,74.6
90.0,3.6,6.5
80.0,0.6,4.6
60.0,0.4,1.2
70.0,0.2,1.7
50.0,0.1,1.5
0.0,0.0,4.9
20.0,0.0,0.9
10.0,NaN,2.7


In [130]:
#hmmm... okay, response time does help slightly
#what about reviews?

superhost = (
    df1[df1['host_is_superhost'] == 't']
    .groupby(round(df1['review_scores_rating']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('Superhost (%)')
)

non_superhost = (
    df1[df1['host_is_superhost'] == 'f']
    .groupby(round(df1['review_scores_rating']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('non_superhost(%)')
)

pd.concat([superhost, non_superhost], axis=1).sort_values(['Superhost (%)'], ascending = False)

,Superhost (%),non_superhost(%)
review_scores_rating,,
5.0,95.1,73.7
4.0,4.5,22.2
3.0,0.2,2.2
1.0,0.1,1.1
2.0,0.1,0.8


In [131]:
superhost = (
    df1[df1['host_is_superhost'] == 't']
    .groupby(round(df1['host_identity_verified']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('Superhost (%)')
)

non_superhost = (
    df1[df1['host_is_superhost'] == 'f']
    .groupby(round(df1['host_identity_verified']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('non_superhost(%)')
)

pd.concat([superhost, non_superhost], axis=1).sort_values(['Superhost (%)'], ascending = False)

,Superhost (%),non_superhost(%)
host_identity_verified,,
t,99.1,98.8
f,0.9,1.2


In [132]:
superhost = (
    df1[df1['host_is_superhost'] == 't']
    .groupby(round(df1['host_has_profile_pic']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('Superhost (%)')
)

non_superhost = (
    df1[df1['host_is_superhost'] == 'f']
    .groupby(round(df1['host_has_profile_pic']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('non_superhost(%)')
)

pd.concat([superhost, non_superhost], axis=1).sort_values(['Superhost (%)'], ascending = False)

,Superhost (%),non_superhost(%)
host_has_profile_pic,,
t,99.2,96.5
f,0.8,3.5
